# 1. Calculating the pressure of vapor saturation

The volatile content of a melt in equilibrium with a vapor can be used as a barometer because the sum of the partial pressures of all the vapor species must equal the total pressure (REF). This is often applied to melt inclusions to calculate magma storage depths (REF) and sub-aqueous matrix glasses to calculate eruption depths (REF).

VolFe can calculate the pressure of vapor saturation (Pvsat), melt speciation, and vapor speciation for a melt of given temperature and melt composition (including volatiles and oxygen fugacity). 

In this example we'll show you how to run this calculation for:

a) a single analysis entered as a dataframe using default options

b) a single analysis from a csv file using default options

c) all analyses in a csv file using default options

d) a set of consequative rows from a csv file using options

e) all analyses in a csv file using one set of user specified options for all analyses

f) all analyses in a csv file using different model options for each analysis

## Setting things up
First we need to import a few Python packages (including VolFe), load the species file, and set the precision of the calculations

In [1]:
# import python packages
import pandas as pd
import gmpy2 as gp
import VolFe as vf

# set calculation precision
gp.get_context().precision=50

## 1a. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for a single melt composition from a dataframe using default options

We need a dataframe of the melt composition, an estimate of oxygen fugacity and Fe in the melt*, and temperature:
- Sample is just the name for this analysis
- Temperature is in 'C
- Volatile-free melt composition is in wt% oxides - all these oxides must be present in the dataframe, so set them to 0. if you have no data for them (although see not around FeOT further down this notebook)
- H2O is all hydrogen in the melt reported as H2O in wt%
- CO2ppm is all carbon in the melt reported as CO2 in ppm
- STppm is all sulfur in the melt reported as S in ppm
- Xppm is all "X" in the melt reported as X in ppm
- Fe3+FeT is the ratio of Fe3+ to FeT in the melt

*in this example we specify oxygen fugacity using Fe3+/FeT and Fe in the melt as FeOT as is quite common for melt inclusion and matrix glass analyses - other options are possible, shown later in the notebook.

The following composition is analysis TN273-01D-01-01 from REF, with a temperature chosen as 1200 'C.

In [2]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeOT': 9.47,
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'Fe3FeT': 0.171}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Show the DataFrame.
print(my_analysis)

            Sample     T_C   SiO2  TiO2  Al2O3  FeOT   MnO   MgO   CaO  Na2O  \
0  TN273-01D-01-01  1200.0  56.98  1.66  15.52  9.47  0.24  2.96  6.49  4.06   

    K2O  P2O5   H2O  CO2ppm   STppm  Xppm  Fe3FeT  
0  0.38  0.22  1.88    13.0  362.83   0.0   0.171  


Now, as we're using default options for everything, we run the calculation like so:

In [3]:
# runs the calculation
vf.calc_Pvsat(my_analysis)

0 TN273-01D-01-01 337.80934489141146


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200.0,337.809345,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:31.139719


Alternatively, we can define the oxygen fugacity using DFMQ and total iron from Fe2O3T.

In [4]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'Fe2O3T': 9.47, ### Fe2O3T instead of FeOT
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'DFMQ': 0.} ### DFMQ instead of Fe3+/FeT

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

0 TN273-01D-01-01 317.21803054879365


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200.0,317.218031,60.242534,1.755047,16.408637,4.504553,0.253742,3.129482,6.861601,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:34.004514


## 1b. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for a single analysis from a csv file using default options

This time, instead of creating a dataframe in a cell for the analysis, we'll read it in from a spreadsheet. We'll use the examples_marianas csv in files and use all the default options again.

In [5]:
# Read csv to define melt composition
my_analyses = pd.read_csv("../files/example_marianas.csv") 

If there was only one analysis in the file, we'd simply run:

vf.calc_Pvsat(my_analysis,species)

However, there are a few so instead we tell the function to stop after the first analysis by saying last_row=1:

In [6]:
# runs the calculation
vf.calc_Pvsat(my_analyses,last_row=1)

0 TN273-01D-01-01 337.80934489141146


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200,337.809345,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:41.809093


## 1c. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for all analyses in a csv file using default options

Using the same example marianas file and the default options, to run all the analyses, simply don't tell it the last row you want to run. All analyses must be in consequitive rows for this to work!


In [7]:
# runs the calculation
vf.calc_Pvsat(my_analyses)

0 TN273-01D-01-01 337.80934489141146
1 TN273-02D-01-02 351.82474110552084
2 Agr07-4 3061.0650112556323
3 Agr07-9 2278.508223859055
4 Agr07-15A 2043.0618685936206
5 Agr07-15B 1570.7780122644162
6 Agr04-13 1841.206481943118
7 Agr04-14 903.6593544856378
8 AGR19-02-4 2198.7509888750064
9 AGR19-02-7 1055.8802476406945
10 AGR19-02-10 2678.8869961665005
11 AGR19-02-11 1938.7223007179382
12 AGR19-02-12B 1745.5601711092277
13 AGR19-02-13 1303.8209208743046
14 AGR19-02-14 2094.864378059443
15 AGR19-02-15 3120.1223410996026
16 AGR19-02-16 2653.958636272426
17 AGR19-02-17 2268.1364385955385
18 Ala02-1 1838.37031211889
19 Ala02-2 1201.6185571777314
20 Ala02-3 2119.5327596207644
21 Ala02-4 2048.198030925887
22 Ala02-5 2905.2093893824385
23 Ala02-7 1797.7932497999227
24 Ala02-8 1810.0092530887705
25 Ala02-9 2059.61466077148
26 Ala02-11 2887.6969430022036
27 Ala02-12 2770.1827964031654
28 Ala02-15 2536.728877728372
29 Ala02-16A 3318.729730031606
30 Ala03-1 1889.3027662407276
31 Sari15-04-3 1342.283023

,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200,337.809345,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:55.704649
0,TN273-02D-01-02,1200,351.824741,56.986918,1.541811,15.548258,9.360992,0.180212,3.434032,6.968183,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:56.049037
0,Agr07-4,1200,3061.065011,45.451495,0.650021,17.270568,10.190335,0.230008,5.540182,12.320405,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:56.652526
0,Agr07-9,1200,2278.508224,48.25551,0.760875,17.309912,9.150526,0.180207,4.825551,12.093912,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:57.247985
0,Agr07-15A,1200,2043.061869,47.769532,0.810841,16.747365,10.681075,0.220228,5.665875,11.34176,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:57.750923
0,Agr07-15B,1200,1570.778012,48.020961,0.860914,16.847879,10.43107,0.190202,5.555896,11.342036,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:58.233135
0,Agr04-13,1200,1841.206482,51.009753,0.801095,16.742895,8.842091,0.200274,4.866655,10.173912,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:58.764688
0,Agr04-14,1200,903.659354,48.748373,0.780775,18.618475,9.319247,0.170169,5.355314,12.071979,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:59.315764
0,AGR19-02-4,1200,2198.750989,47.941949,0.655476,17.667136,9.729727,0.266287,4.700994,13.099285,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:39:59.876533
0,AGR19-02-7,1200,1055.880248,47.730139,0.852868,18.570182,9.472925,0.172604,4.406484,12.853936,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:00.300013


## 1d. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for  a set of consequative rows from a csv file using options

If we only want to run row 4 (Agr07-4) through 18 (AGR19-02-16) in that spreadsheet, we simply specify the first and last rows we want to run (remembering that row 2 in a spreadsheet - i.e., the first analysis under the headings - is actually row 0 in the dataframe).

In [8]:
vf.calc_Pvsat(my_analyses,first_row=2, last_row=17)

2 Agr07-4 3061.0650112556323
3 Agr07-9 2278.508223859055
4 Agr07-15A 2043.0618685936206
5 Agr07-15B 1570.7780122644162
6 Agr04-13 1841.206481943118
7 Agr04-14 903.6593544856378
8 AGR19-02-4 2198.7509888750064
9 AGR19-02-7 1055.8802476406945
10 AGR19-02-10 2678.8869961665005
11 AGR19-02-11 1938.7223007179382
12 AGR19-02-12B 1745.5601711092277
13 AGR19-02-13 1303.8209208743046
14 AGR19-02-14 2094.864378059443
15 AGR19-02-15 3120.1223410996026
16 AGR19-02-16 2653.958636272426


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,Agr07-4,1200,3061.065011,45.451495,0.650021,17.270568,10.190335,0.230008,5.540182,12.320405,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:25.314310
0,Agr07-9,1200,2278.508224,48.25551,0.760875,17.309912,9.150526,0.180207,4.825551,12.093912,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:25.849497
0,Agr07-15A,1200,2043.061869,47.769532,0.810841,16.747365,10.681075,0.220228,5.665875,11.34176,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:26.358869
0,Agr07-15B,1200,1570.778012,48.020961,0.860914,16.847879,10.43107,0.190202,5.555896,11.342036,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:26.812473
0,Agr04-13,1200,1841.206482,51.009753,0.801095,16.742895,8.842091,0.200274,4.866655,10.173912,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:27.276298
0,Agr04-14,1200,903.659354,48.748373,0.780775,18.618475,9.319247,0.170169,5.355314,12.071979,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:27.723198
0,AGR19-02-4,1200,2198.750989,47.941949,0.655476,17.667136,9.729727,0.266287,4.700994,13.099285,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:28.234313
0,AGR19-02-7,1200,1055.880248,47.730139,0.852868,18.570182,9.472925,0.172604,4.406484,12.853936,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:28.632892
0,AGR19-02-10,1200,2678.886996,47.019198,0.74585,18.196695,9.185193,0.153257,4.689659,12.914443,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:29.232938
0,AGR19-02-11,1200,1938.722301,47.917169,0.777976,18.077701,9.315237,0.214967,4.729274,12.713763,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:29.737055


## 1e. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for all analyses in a csv file using one set of user specified options for all analyses

What if I don't want to use the default options? In that case, we create a dataframe telling VolFe what to use instead. There are lots of options that can be changed (see models csv in files for the full list, description, and options currently available). But let's say I just want to use a different solubility constant for carbon dioxide and hydrogen sulfide and treat S2 as an ideal gas. Also I don't want it to print the status during a calculation.

In [9]:
# choose the options I want - everything else will use the default options
my_models = [['carbon dioxide','Dixon97'],['hydrogen sulfide','basaltic andesite'],['y_S2','ideal'],['print status','no']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

# show what the model dataframe looks like
print(my_models)

                option
type                  
insolubles         yes
H2S_m              yes
species X           Ar
Hspeciation       none
fO2           Kress91A
...                ...
bulk_O           exc S
error              0.1
print status        no
output csv         yes
setup               no

[62 rows x 1 columns]


In [10]:
vf.calc_Pvsat(my_analyses,models=my_models)

,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200,287.638259,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:46.463288
0,TN273-02D-01-02,1200,220.240536,56.986918,1.541811,15.548258,9.360992,0.180212,3.434032,6.968183,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:46.786337
0,Agr07-4,1200,2933.114684,45.451495,0.650021,17.270568,10.190335,0.230008,5.540182,12.320405,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:47.307165
0,Agr07-9,1200,2034.275247,48.25551,0.760875,17.309912,9.150526,0.180207,4.825551,12.093912,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:47.852998
0,Agr07-15A,1200,1661.452078,47.769532,0.810841,16.747365,10.681075,0.220228,5.665875,11.34176,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:48.402120
0,Agr07-15B,1200,1406.19412,48.020961,0.860914,16.847879,10.43107,0.190202,5.555896,11.342036,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:48.842306
0,Agr04-13,1200,1873.065683,51.009753,0.801095,16.742895,8.842091,0.200274,4.866655,10.173912,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:49.368844
0,Agr04-14,1200,818.456709,48.748373,0.780775,18.618475,9.319247,0.170169,5.355314,12.071979,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:49.795531
0,AGR19-02-4,1200,1664.283666,47.941949,0.655476,17.667136,9.729727,0.266287,4.700994,13.099285,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:50.278546
0,AGR19-02-7,1200,941.372237,47.730139,0.852868,18.570182,9.472925,0.172604,4.406484,12.853936,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:40:50.666984


Alternatively, you can load a csv with the model options listed in it (see the example models csv in the files).

In [11]:
my_models = pd.read_csv("../files/models.csv", index_col = [0])
vf.calc_Pvsat(my_analyses,models=my_models)

0 TN273-01D-01-01 337.80934489141146
1 TN273-02D-01-02 351.82474110552084
2 Agr07-4 3061.0650112556323
3 Agr07-9 2278.508223859055
4 Agr07-15A 2043.0618685936206
5 Agr07-15B 1570.7780122644162
6 Agr04-13 1841.206481943118
7 Agr04-14 903.6593544856378
8 AGR19-02-4 2198.7509888750064
9 AGR19-02-7 1055.8802476406945
10 AGR19-02-10 2678.8869961665005
11 AGR19-02-11 1938.7223007179382
12 AGR19-02-12B 1745.5601711092277
13 AGR19-02-13 1303.8209208743046
14 AGR19-02-14 2094.864378059443
15 AGR19-02-15 3120.1223410996026
16 AGR19-02-16 2653.958636272426
17 AGR19-02-17 2268.1364385955385
18 Ala02-1 1838.37031211889
19 Ala02-2 1201.6185571777314
20 Ala02-3 2119.5327596207644
21 Ala02-4 2048.198030925887
22 Ala02-5 2905.2093893824385
23 Ala02-7 1797.7932497999227
24 Ala02-8 1810.0092530887705
25 Ala02-9 2059.61466077148
26 Ala02-11 2887.6969430022036
27 Ala02-12 2770.1827964031654
28 Ala02-15 2536.728877728372
29 Ala02-16A 3318.729730031606
30 Ala03-1 1889.3027662407276
31 Sari15-04-3 1342.283023

,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200,337.809345,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:24.745284
0,TN273-02D-01-02,1200,351.824741,56.986918,1.541811,15.548258,9.360992,0.180212,3.434032,6.968183,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:25.110064
0,Agr07-4,1200,3061.065011,45.451495,0.650021,17.270568,10.190335,0.230008,5.540182,12.320405,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:25.664081
0,Agr07-9,1200,2278.508224,48.25551,0.760875,17.309912,9.150526,0.180207,4.825551,12.093912,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:26.166483
0,Agr07-15A,1200,2043.061869,47.769532,0.810841,16.747365,10.681075,0.220228,5.665875,11.34176,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:26.664092
0,Agr07-15B,1200,1570.778012,48.020961,0.860914,16.847879,10.43107,0.190202,5.555896,11.342036,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:27.114728
0,Agr04-13,1200,1841.206482,51.009753,0.801095,16.742895,8.842091,0.200274,4.866655,10.173912,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:27.567118
0,Agr04-14,1200,903.659354,48.748373,0.780775,18.618475,9.319247,0.170169,5.355314,12.071979,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:28.019262
0,AGR19-02-4,1200,2198.750989,47.941949,0.655476,17.667136,9.729727,0.266287,4.700994,13.099285,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:28.520010
0,AGR19-02-7,1200,1055.880248,47.730139,0.852868,18.570182,9.472925,0.172604,4.406484,12.853936,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-03-04 20:41:28.920989


## 1f. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for  all analyses in a csv file using different model options for each analysis

